In [ ]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

In [ ]:
class Value:

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f'Value(data={self.data})'
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other) # for if being added to a non Value type bc no .data

        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        
        out._backward = _backward # stores function instead of calling for value (_background()) which is type none
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)

        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out
    
    def __pow__(self, other):
        #checking to make sure its int or float
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad # chained
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other (self * other**-1)
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)
    
    def __radd__(self, other): # other + self
        return self + other

    # Activation function
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1-t**2) * out.grad

        out._backward = _backward

        return out
    
    # def exp(self): # e^x
    #     x = self.data
    #     out = Value(math.exp(x), (self, ), 'exp')

    #     def _backward():
    #         self.grad += out.data * out.grad
    #     out._backward = _backward

    #     return out
    
    # topological sort and then back propigate using _backward stored functions through them in order.
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [ ]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self, x): # if n = Neuron(2) and x = [1,2] , n(x) calls this func and iterates through all x
        # w * x + b | zip(self.w, x) is tuple (weight, x)
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b) # end self.b is just + self.b
        # forward activation
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b] # list of w concatinated with list of b, w + b, list + list

class Layer:

    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range (nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
        # Same code
        # params = []
        # for neuron in self.neurons:
        #     ps = neuron.parameters()
        #     params.extend(ps)
        # return params
    
class MLP: # mlp = MLP(inputs, [l1, l2, out])

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [ ]:
n = MLP(3, [4,4,1])

xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [ ]:
# forward pass
ypred = [n(x) for x in xs]
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

In [ ]:
# back prop
loss.backward()

In [ ]:
n.layers[0].neurons[0].w[0].grad

In [ ]:
n.layers[0].neurons[0].w[0].data

In [ ]:
# nudge, SGD, update
for p in n.parameters():
    p.data += -0.01 * p.grad # param - lr * grad

In [ ]:
n = MLP(3, [4,4,1])

xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [ ]:
for k in range(20): # x steps

    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    # backward pass

    #zero-grad
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    #update
    for p in n.parameters():
        p.data += -0.05 * p.grad # param - lr * grad

    print(k, loss.data)

In [ ]:
ypred

What you can add:
- Learning rate decay lr gets finer as loss gets smaller
- momentum?
- ReLU
- diff thing than SGD

In [ ]:
min = -np.pi
max = np.pi

steps = 1000
batch_size = 100
log_every_steps = 1

learning_rate = 1e-3

rates = []

# class NeuralNetwork():
#     def __init__(self):
#         self.NN = MLP(1, [10, 10, 1]) # 1 in 10 hidden 1 out (maybe 2nd 10 hidden)
    # def forward(self, batch): # batch of x's
    #     ypred = [n(n) for n in batch]
    #     return ypred

model = MLP(1, [5, 5, 1])

for i in range(steps):
    # Gen batch
    x = np.random.rand(batch_size, 1) * 2 * max - min
    
    # Forward
    y_expected = np.sin(x)
    y_hat = [model(i) for i in x]

    loss = sum((yout - ygt)**2 for ygt, yout in zip(y_expected, y_hat))

    # Backward
    for p in model.parameters():
        p.grad = 0.0
    loss.backward()

    # Optimizer Update
    for p in model.parameters():
        p.data += -learning_rate * p.grad
    
    if i > 0 and i % log_every_steps == 0:
        print(f'batch: {i} | loss: {loss.data}')
        rates.append(loss.data)

plt.plot(rates)